In [1]:
import importlib
import os
import numpy as np
import pandas as pd
import preprocessing as preproc
import nltk
import joblib

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

importlib.reload(preproc)


<module 'preprocessing' from 'c:\\Users\\kubia\\NLP-AI-Case-Studies\\preprocessing.py'>

In [51]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kubia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

Use preproc.prepare_imdb_data_csv only if you dont wanna use already prepared csv file

In [ ]:
df = preproc.prepare_imdb_data_csv()

In [30]:
df = pd.read_csv('sentiment_imdb_data.csv')

In [31]:
df = df[['Review', 'Sentiment']]
df

,Review,Sentiment
0,Over-powered mobile suits that can annihilate ...,1
1,This is a terrible movie that only gets worse ...,0
2,"This is an early film ""Pilot"" for the hit Cana...",1
3,Family Guy is THE best show on TV. EVER. It ha...,1
4,I was China in this film. I choose the screen ...,0
...,...,...
49995,STAR RATING: ***** Saturday Night **** Friday ...,0
49996,Pretty good movie about a man and his wife who...,1
49997,"It was once suggested by Pauline Kael, never a...",1
49998,"The film notes describe the main role family, ...",1


## Przekształcanie słów w wektory cech

Przykład prezentujacy działanie klasy CountVectorizer

In [35]:
sentences = [
    "Codziennie pracuję przy komputerze w biurze.",
    "Moje biuro jest nowoczesne i dobrze wyposażone.",
    "Praca przy komputerze może być męcząca.",
    "W biurze mamy spotkania codziennie rano.",
    "Komputer w moim biurze jest bardzo szybki.",
    "Lubię robić zakupy w dużych centrach handlowych.",
    "W sklepie często znajduję promocje i zniżki.",
    "Zakupy online są bardzo wygodne i szybkie.",
    "Codziennie sprawdzam oferty w sklepach internetowych.",
    "Moje ulubione sklepy mają zawsze szeroki wybór produktów.",
    "Codziennie biegam w parku, aby utrzymać formę.",
    "Bieganie jest moim ulubionym sposobem na relaks.",
    "W parku często spotykam innych biegaczy.",
    "Mój trener zaleca bieganie codziennie.",
    "Codzienne bieganie pomaga mi w utrzymaniu zdrowia.",
    "Komputer jest w moim biurze. Komputer działa bardzo szybko. Komputer jest nowoczesny.",
    "Zakupy w sklepie są wygodne. Zakupy online są szybkie. Zakupy to moja ulubiona czynność.",
    "Bieganie jest zdrowe. Bieganie codziennie poprawia kondycję. Bieganie w parku jest relaksujące."
] # wygenerowane przy pomocy ChatGPT
sentences_array = np.array(sentences)

Reprezentacja unigramowa:

In [36]:
countVec = CountVectorizer()
bag = countVec.fit_transform(sentences_array)
print(bag.toarray()[0], '\nLiczba słów reprezentowana przez wektor: ' ,bag.toarray()[0].sum())
print(countVec.vocabulary_)

[0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0] 
Liczba słów reprezentowana przez wektor:  5
{'codziennie': 10, 'pracuję': 43, 'przy': 46, 'komputerze': 22, 'biurze': 6, 'moje': 30, 'biuro': 5, 'jest': 20, 'nowoczesne': 35, 'dobrze': 13, 'wyposażone': 72, 'praca': 42, 'może': 31, 'być': 7, 'męcząca': 33, 'mamy': 26, 'spotkania': 55, 'rano': 47, 'komputer': 21, 'moim': 28, 'bardzo': 1, 'szybki': 59, 'lubię': 24, 'robić': 50, 'zakupy': 73, 'dużych': 14, 'centrach': 8, 'handlowych': 17, 'sklepie': 52, 'często': 12, 'znajduję': 78, 'promocje': 45, 'zniżki': 79, 'online': 38, 'są': 62, 'wygodne': 71, 'szybkie': 60, 'sprawdzam': 57, 'oferty': 37, 'sklepach': 51, 'internetowych': 19, 'ulubione': 66, 'sklepy': 53, 'mają': 25, 'zawsze': 75, 'szeroki': 58, 'wybór': 70, 'produktów': 44, 'biegam': 3, 'parku': 39, 'aby': 0, 'utrzymać': 69, 'formę': 16, 'bieganie': 4, 'ulubionym': 67, 

Reprezentacja 2-gramowa:

In [37]:
countVec2 = CountVectorizer(ngram_range=(2,2))
bag2 = countVec2.fit_transform(sentences_array)
print(bag2.toarray()[0], '\nLiczba słów reprezentowana przez wektor: ' ,bag2.toarray()[0].sum())
print(countVec2.vocabulary_)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 
Liczba słów reprezentowana przez wektor:  4
{'codziennie pracuję': 18, 'pracuję przy': 60, 'przy komputerze': 62, 'komputerze biurze': 36, 'moje biuro': 46, 'biuro jest': 9, 'jest nowoczesne': 29, 'nowoczesne dobrze': 51, 'dobrze wyposażone': 23, 'praca przy': 59, 'komputerze może': 37, 'może być': 48, 'być męcząca': 13, 'biurze mamy': 12, 'mamy spotkania': 41, 'spotkania codziennie': 69, 'codziennie rano': 19, 'komputer moim': 35, 'moim biurze': 43, 'biurze jest': 10, 'jest bardzo': 27, 'bardzo szybki': 1, 'lubię robić': 39, 'robić zakupy': 63, 'zakupy dużych': 88, 'dużych centrach': 24, 'centrach handlowych': 14, 'sklepie często': 65, 'często znajduję': 22, 'znajduję promocje': 95, 'promocje zniżki': 61, 'zakupy online': 89, 'online są': 53, 'są bardzo': 75, 'bardzo wygodne': 3, 'wygodne sz

Reprezentację, którą należy wybrać zależy od typu zadania z którym się borykamy, 
przykładowo prof. Kanaris dowiedli że n-gramy o rozmiarze 3 lub 4 dają bardzo dobre wyniki dla rozwiązania problemu wykrycia spamu poczty e-mail.

----

Słowa często występujące w wielu różnych danych tekstowych bardzo często nie niosą za sobą rozróżniających informacji. Aby zmniejszyć wpływ takich słów na uczenie modelu możemy wykorzystać **ważenie częstości termów**

In [38]:
tfidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf=True)
tfidf.fit_transform(countVec.fit_transform(sentences_array)).toarray()[-1]

array([0.        , 0.        , 0.        , 0.        , 0.63832904,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.1821154 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.39232468, 0.        , 0.        , 0.29627309, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.23311024,
       0.        , 0.29627309, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.29627309,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [39]:
print(df[df['Review'].str.contains(r":\)")]['Review'].iloc[1])

One reasons why they call the 80's, "The Awesome 80's" is quality television. Shows like the Wonder Years, War of the Worlds (the series), V, Amazing Stories, and many more have always left an impression to each "fortunate" one of us that in time will always find a way to reawaken itself. To top that, here comes Monsters! A series quite unique of its own, and a theme fully dedicate to - monsters. May it be the good, the bad, and the morbid.<br /><br />If you're a fan of classic shows or if you have the fascination of horror films then this one is absolutely for you. Provided you can find this rare gem. <br /><br />Even the newer generations will be in awe with some of the episode with its grittiness, it's indiscriminating use of gore effects or its story telling power and simplicity. I guarantee, because I'm 23 :).<br /><br />Be sure NOT to miss this!<br /><br />Although, it's a show seemingly forgotten by the modern world, it will always be with those who can always remember...


In [54]:
t = preproc.regex_preprocessing(df[df['Review'].str.contains(r":\)")]['Review'].iloc[0])
t

'this movie is a perfect example of barkers cinematic gifts to the horror monster genre i thought this movie did a great job of keeping the feel and look of the novella and comic books or actually the comics may have come second i forget this movie was made for barker fans it helps to have read the book beforehand but isn t that important if you can follow a film i saw to anyone who is on the fence about this film read the book then re watch the film you might find a new respect for the movie i came to this movie a big fan of barker already and having read the book prior loved the film instantly there are great cameos makeup writing directing etc in this film this movie does something that most monster horror movies fail miserably at show the monsters they are there in full color not hidden in shadows and taking most of the screen time unlike other films that use quick cuts or trick lighting to hide the creature this movie celebrates the grotesque and casts them into the forefront as t

In [41]:
df['Review'] = df['Review'].apply(preproc.regex_preprocessing)

In [55]:
df['Review'].iloc[0]

'i went and saw this movie last night after being coaxed to by a few friends of mine i ll admit that i was reluctant to see it because from what i knew of ashton kutcher he was only able to do comedy i was wrong kutcher played the character of jake fischer very well and kevin costner played ben randall with such professionalism the sign of a good movie is that it can toy with our emotions this one did exactly that the entire theater which was sold out was overcome by laughter during the first half of the movie and were moved to tears during the second half while exiting the theater i not only saw many women in tears but many full grown men as well trying desperately not to let anyone see them crying this movie was great and i suggest that you go see it before you judge'

In [53]:
preproc.tokenize_porter(df['Review'].iloc[0])

['went',
 'saw',
 'movi',
 'last',
 'night',
 'coax',
 'friend',
 'mine',
 'admit',
 'reluct',
 'see',
 'knew',
 'ashton',
 'kutcher',
 'abl',
 'comedi',
 'wrong',
 'kutcher',
 'play',
 'charact',
 'jake',
 'fischer',
 'well',
 'kevin',
 'costner',
 'play',
 'ben',
 'randal',
 'profession',
 'sign',
 'good',
 'movi',
 'toy',
 'emot',
 'one',
 'exactli',
 'entir',
 'theater',
 'sold',
 'overcom',
 'laughter',
 'first',
 'half',
 'movi',
 'move',
 'tear',
 'second',
 'half',
 'exit',
 'theater',
 'saw',
 'mani',
 'women',
 'tear',
 'mani',
 'full',
 'grown',
 'men',
 'well',
 'tri',
 'desper',
 'let',
 'anyon',
 'see',
 'cri',
 'movi',
 'great',
 'suggest',
 'go',
 'see',
 'judg']

In [56]:
df

,Review,Sentiment
0,i went and saw this movie last night after bei...,1
1,actor turned director bill paxton follows up h...,1
2,as a recreational golfer with some knowledge o...,1
3,i saw this film in a sneak preview and it is d...,1
4,bill paxton has taken the true story of the 19...,1
...,...,...
49995,towards the end of the movie i felt it was too...,0
49996,this is the kind of movie that my enemies cont...,0
49997,i saw descent last night at the stockholm film...,0
49998,some films that you pick up for a pound turn o...,0


In [66]:
np.random.seed(42)
df = df.reindex(np.random.permutation(df.index)).reset_index(drop=True)

In [80]:
X_train, X_test, y_train , y_test = train_test_split(df['Review'], df['Sentiment'], test_size=0.35, random_state=42, stratify=df['Sentiment'])

In [81]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
stop_words = stopwords.words('english')
param_grid = [{'vect__ngram_range': [(1,1), (2,2)],
               'vect__tokenizer': [preproc.tokenize,
                                    preproc.tokenize_porter],
               'vect__stop_words': [stop_words,None],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 50.0]},

                {'vect__ngram_range': [(1,1), (2,2)],
               'vect__tokenizer': [preproc.tokenize,
                                    preproc.tokenize_porter],
               'vect__stop_words': [stop_words,None],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 50.0],
                'vect__use_idf': [False],
                'vect__norm': [None]}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=42))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv = 5, verbose=1, n_jobs = -1)

gs_lr_tfidf.fit(X_train,y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


c:\Users\kubia\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\kubia\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\kubia\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kubia\App

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 50.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1), (2, 2)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "y...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenize at 0x000002CBBF177880>,
                                              <function tokenize_porter at 0x000002CBDA186840>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=1)

In [79]:
print(X_train.shape, y_train.shape)

(32500,) (17500,)


In [82]:
gs_lr_tfidf.best_params_

{'clf__C': 50.0,
 'clf__penalty': 'l2',
 'vect__ngram_range': (2, 2),
 'vect__stop_words': None,
 'vect__tokenizer': <function preprocessing.tokenize(text)>}

In [83]:
gs_lr_tfidf.best_score_

np.float64(0.8980615384615385)

In [84]:
model = gs_lr_tfidf.best_estimator_

In [106]:
model.score(X_test,y_test)

0.9012

In [107]:
model.predict_proba([preproc.regex_preprocessing('Deadpool & Wolverine is enjoyable on its merits: R-rated, horribly violent juvenile fantasy loaded with nostalgic references from the glory days of comic reading that fans, new and old, will thoroughly enjoy as it drags you down to its irreverently funny level.')])

array([[0.26608429, 0.73391571]])

In [110]:


joblib.dump(model, 'sentiment_model.joblib')

['sentiment_model.joblib']

-------------------------

# Optymalizacja procesu uczenia za pomocą uczenia pozardzeniowego

In [3]:
#df = pd.read_csv('sentiment_imdb_data.csv')
#np.random.seed(1042)
#df = df.reindex(np.random.permutation(df.index))
#df['Sentiment'].value_counts()

poczytaj o yield i next

In [2]:
csv_path = 'sentiment_imdb_data.csv'
import csv
def new_tokenizer(text):
    stop_words = stopwords.words('english')
    text_array = [word for word in preproc.tokenize(preproc.regex_preprocessing(text)) if word not in stop_words]

    return text_array

def stream_csv(path):
    with open(os.path.join(path), 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            text, label = row['Review'], int(row['Sentiment']) 
            yield text, label

#next(stream_csv(csv_path))

csv_stream = stream_csv(csv_path)

def get_minibatch_from_file(stream_csv, batch_size):

    X, y = [], []
    try:
        for _ in range(batch_size):
            text, label = next(stream_csv)
            X.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return X,y

#get_minibatch_from_file(csv_stream, 12)

In [3]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from tqdm import tqdm
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=new_tokenizer)

num_training_samples = 45000
batch_size = 1000

clf = SGDClassifier(loss='log_loss', random_state=42, max_iter=1)

labels = np.array([0,1])

for _ in tqdm(range(num_training_samples//batch_size), desc= 'Traning phase'):
    X_train, y_train = get_minibatch_from_file(csv_stream, batch_size=batch_size)
    if not X_train:
        break

    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=labels)

remaining_samples = sum(1 for _ in csv_stream)
if remaining_samples < 5000:
    raise ValueError("Not enough samples left for testing after training.")

csv_stream = stream_csv(csv_path)

for _ in range(num_training_samples):
    next(csv_stream)

X_test, y_test = get_minibatch_from_file(csv_stream, 5000)
X_test = vect.transform(X_test)
score = clf.score(X_test, y_test)

print(f'Test Accuracy: {score:.4f}')

clf = clf.partial_fit(X_test, y_test)

Traning phase: 100%|██████████| 45/45 [00:16<00:00,  2.67it/s]


Test Accuracy: 0.8682


In [6]:
joblib.dump(clf, 'improved_sentiment_model.joblib')

['improved_sentiment_model.joblib']

In [19]:
result = clf.predict_proba(vect.transform(['When it debuted in 1991, Terminator 2: Judgment Day blew the collective minds of action movie junkies and, well, everyone who enjoys movies. What size this production has, what innovation, and what crowd-pleasing excitement. Marked by its breakthrough special effects, an unending series of bravura action sequences, and clever narrative inversion of the 1984 original, the production sells itself: Arnold Schwarzenegger, then at the height of his popularity, plays the good guy this time around, protecting the future’s rebel leader, John Connor, played by newcomer Edward Furlong. Linda Hamilton committed to her transformation into a ferocious survivalist and earned Oscar buzz for her performance. The new CGI technology renders an iconic movie villain, played by Robert Patrick, composed of liquid metal. And director James Cameron, in his prime after Aliens (1986) and The Abyss (1989), takes the scope far beyond what he achieved in the first, offering unexpected emotional value on top of his breakneck spectacle. T2 was the most expensive film ever made then, and it outperformed all box-office projections. This would be a trend for Cameron, which he would repeat with Titanic (1997) and his Avatar films. In its day, there was hardly a film more talked about or at the forefront of the zeitgeist than T2, and it would change the way blockbusters were made for the foreseeable future.']))

print(1 if result[0][1]> 0.5 else 0)

1


In [32]:
df['Review'] = df['Review'].apply(preproc.regex_preprocessing).reset_index(drop = True)

In [33]:
df

,Review,Sentiment
0,over powered mobile suits that can annihilate ...,1
1,this is a terrible movie that only gets worse ...,0
2,this is an early film pilot for the hit canadi...,1
3,family guy is the best show on tv ever it has ...,1
4,i was china in this film i choose the screen n...,0
...,...,...
49995,star rating saturday night friday night friday...,0
49996,pretty good movie about a man and his wife who...,1
49997,it was once suggested by pauline kael never a ...,1
49998,the film notes describe the main role family a...,1


In [35]:
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)

X_lda = count.fit_transform(df['Review'].values)

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components = 10, random_state=123, learning_method='batch')

X_topics = lda.fit_transform(X_lda)


In [37]:
lda.components_

array([[1.23347744e+01, 9.33925051e+01, 1.14651100e+02, ...,
        1.16974583e+03, 8.20468598e+02, 3.99462945e+01],
       [3.64806835e+00, 6.59362417e+00, 7.26364571e+01, ...,
        1.00004698e-01, 1.00003597e-01, 6.08081320e-01],
       [1.28972916e+00, 8.90783791e+01, 3.73251971e+01, ...,
        1.00004863e-01, 1.00006898e-01, 3.69256345e+01],
       ...,
       [3.83353204e+01, 2.52100889e+02, 5.57809432e+01, ...,
        1.43499961e-01, 2.91402865e-01, 1.24591309e+01],
       [6.06510486e+01, 5.94358788e+01, 2.62038999e+02, ...,
        5.01115935e+00, 1.77377000e+00, 1.26556788e+01],
       [9.40935102e+00, 4.62148700e+01, 1.07270256e+02, ...,
        1.00007048e-01, 1.00007233e-01, 1.00010660e-01]])

In [36]:
n_top_words = 10
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print("Temat %d.:" % (topic_idx+1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words -1:-1]]))

Temat 1.:
horror effects budget low gore special blood scary dead monster
Temat 2.:
sex girl guy women feel kind maybe woman girls sure
Temat 3.:
war american history political german men country soldiers british government
Temat 4.:
wife father woman performance role mother murder husband plays son
Temat 5.:
series tv episode kids dvd comedy episodes remember shows watched
Temat 6.:
comedy star role original performance excellent fun effects wonderful fi
Temat 7.:
music beautiful family human cinema true art documentary children wonderful
Temat 8.:
town john car guy plays city goes jack later girl
Temat 9.:
worst minutes book script awful waste read terrible money stupid
Temat 10.:
action game music musical song version songs dvd dance fight


In [38]:
horror = X_topics[:, 0].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n')
    print(df['Review'][movie_idx][:300], '...')



title zombie 3 1988 directors mostly lucio fulci but also claudio fragasso and bruno mattei cast ottaviano dellacqua massimo vani beatrice ring deran serafin review to review this flick and get some good background of it i gotta start by the beginning and the beginning of this is really george romer ...


over christmas break a group of college friends stay behind to help prepare the dorms to be torn down and replaced by apartment buildings to make the work a bit more difficult a murderous chucks wearing psycho is wandering the halls of the dorm preying on the group in various violent ways registered ...


fulci does this man brings one of the goriest and weirdest movies ever made answer yes cat in the brain also known as nightmare concert is fulci s last masterpiece yes it is no matter what some people will say about it there are few facts why this movie is one of the best fulci s movies fulci make a ...


In [40]:
new_text = ["title zombie 3 1988 directors mostly lucio fulci but also claudio fragasso and bruno mattei cast ottaviano dellacqua massimo vani beatrice ring deran serafin review to review this flick and get some good background of it i gotta start by the beginning and the beginning of this is really george romer"]  # Example new text


new_X_lda = count.transform(new_text)

new_X_topics = lda.transform(new_X_lda)

assigned_topic = new_X_topics.argmax(axis=1)[0]

print("Assigned Topic:", assigned_topic + 1)
print("Topic Distribution:", new_X_topics[0])

Assigned Topic: 1
Topic Distribution: [0.94704468 0.00588364 0.00588368 0.00588474 0.00588393 0.00588369
 0.00588362 0.00588387 0.00588413 0.00588402]
